Make sure runtime is GPU

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
!pip install flair
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from flair.models import SequenceTagger
SequenceTagger.load('chunk')
SequenceTagger.load('pos')

     |████████████████████████████████| 337kB 4.5MB/s 
     |████████████████████████████████| 1.1MB 10.2MB/s 
     |████████████████████████████████| 983kB 18.7MB/s 
     |████████████████████████████████| 276kB 31.8MB/s 
     |████████████████████████████████| 19.7MB 1.5MB/s 
     |████████████████████████████████| 798kB 49.0MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 1.3MB 47.6MB/s 
     |████████████████████████████████| 2.9MB 48.6MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=c2d22fcaa565a0635e56e8b5178a75d99ee29ac045c2aeaf45a282a704e6e4bb
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25021 sha256=691d00b29c281c4708ba0ca361ac92fe8fb36df3f86f635e03315e02d3a563db
  Stored in directory: 

100%|██████████| 249034168/249034168 [00:40<00:00, 6130504.33B/s]

2020-10-24 19:08:37,015 copying /tmp/tmpnk_y9mnq to cache at /root/.flair/models/en-chunk-conll2000-v0.4.pt


2020-10-24 19:08:37,544 removing temp file /tmp/tmpnk_y9mnq
2020-10-24 19:08:38,349 loading file /root/.flair/models/en-chunk-conll2000-v0.4.pt
2020-10-24 19:08:53,176 https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt not found in cache, downloading to /tmp/tmpi84mrec4


100%|██████████| 249072763/249072763 [00:15<00:00, 16165666.10B/s]

2020-10-24 19:09:09,058 copying /tmp/tmpi84mrec4 to cache at /root/.flair/models/en-pos-ontonotes-v0.5.pt


2020-10-24 19:09:09,714 removing temp file /tmp/tmpi84mrec4
2020-10-24 19:09:09,773 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=53, bias=True)
)

In [3]:
import re

In [4]:
from flair.data import Sentence
from nltk import tokenize, sent_tokenize
import pandas as pd
from tqdm import tqdm
import logging

# Filtering
1. First use chunking to find sentences that have at least three noun phrases (A, B, pronoun)
2. Then use POS tagging to find sentences that have the specific pronouns you want


In [5]:
tagger = SequenceTagger.load('chunk')
ner_tagger = SequenceTagger.load('ner')
pos_tagger = SequenceTagger.load('pos')

2020-10-24 19:09:23,880 loading file /root/.flair/models/en-chunk-conll2000-v0.4.pt
2020-10-24 19:09:25,175 https://nlp.informatik.hu-berlin.de/resources/models/ner/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmpajheaui6


100%|██████████| 432197603/432197603 [00:24<00:00, 17663784.10B/s]

2020-10-24 19:09:50,118 copying /tmp/tmpajheaui6 to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2020-10-24 19:09:51,487 removing temp file /tmp/tmpajheaui6
2020-10-24 19:09:51,534 loading file /root/.flair/models/en-ner-conll03-v0.4.pt
2020-10-24 19:09:53,241 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


In [6]:
def get_sentences(raw_sentences,tagger,ner_tagger,pos_tagger,search_pattern):
  # print('Tokenizing sentences for filter 1')
  sentences = [Sentence(s) for s in raw_sentences]

  # Chunking the sentences into phrases
  # tagger = SequenceTagger.load('chunk')
  # print("Chunking sentences")
  tagger.predict(sentences)

  # Count the number of noun phrases. There should be at least 3 (A, B, pronoun)
  # logging.info("\nApplying filter 1")
  filter1_sentences = []
  for sent in sentences:
      if [chunk.tag for chunk in sent.get_spans()].count('NP') >= 3:
          filter1_sentences.append(sent.to_original_text())
  # print('\nFilter 1: Removed {} sentences'.format(len(sentences) - len(filter1_sentences)))

  # print("Tokenizing sentences for NER filter")
  sentences = [Sentence(s) for s in filter1_sentences]
  # ner_tagger = SequenceTagger.load('ner')
  ner_tagger.predict(sentences)

  # logging.info("\nApplying filter NER")
  NER_sentences = []
  for sent in sentences:
      if [entity.tag for entity in sent.get_spans('ner')].count('PER') >= 2:
        for entity in sent.get_spans('ner'):
          if entity.tag == 'PER' and len(entity)> 1:
            original_name = entity.text
            new_name = str(entity.text).split(' ')[0] # gets the first name
            sent = sent.to_original_text()
            sent = Sentence(sent.replace(original_name, new_name)) #Replaces the name and re-tokenizes the sentence
        
        NER_sentences.append(sent.to_original_text())

  # print('\n Sentences after Filter 1: {}'.format(len(NER_sentences)))
  # print("Tokenizing sentences for filter 2")
  sentences = [Sentence(s) for s in NER_sentences]
  len(sentences)
  # POS tagging
  # pos_tagger = SequenceTagger.load('pos')
  # print("Tagging sentences with POS")
  pos_tagger.predict(sentences)

  # Only select sentences with one gendered pronoun and no other pronouns
  # print("\nApplying filter 2")
  filter2_sentences = []
  for sent in sentences:
      token_list = [token.text for token in sent]
      if (token_list.count('he') + token_list.count('she') == 1) and [token.get_tag('pos').value for token in sent].count('PRP') == 1:
          filter2_sentences.append(sent.to_original_text())
  print("len after filter 2 is {}".format(len(filter2_sentences)))

  filter3 = []
  for sent in filter2_sentences:
    r = search_pattern.search(sent)
    if r:
      print("RE match found {}".format(r.start()))
      sentence = Sentence(sent[:r.start()])
      ner_tagger.predict(sentence)
      if [entity.tag for entity in sentence.get_spans('ner')].count('PER') >= 1:
        filter3.append(sent)


  # print('\nFilter 2: Removed {} sentences'.format(len(sentences) - len(filter2_sentences)))
  print("length after filter 3 is {}".format(len(filter3)))
  return filter3

In [ ]:
wiki_san = pd.read_csv('/gdrive/My Drive/NLP-datasets/movie_reviews.csv',names=['Article','Content','link'],encoding='cp1252')
wiki_san['Content']




# Extract He/she

In [ ]:
# Get raw sentences from the scrape, removes non-ASCII characters, fixes the full-stop issue 
filter2_sentences_extract = []
total_sentences = 0
search_pattern = re.compile(r"( he | she )")
stop_regex = re.compile(r"\.")
no_bracket_regex = re.compile(r"[\(\[].*?[\)\]]")
for para in tqdm(wiki_san['Content']):
  try:
    para = stop_regex.sub(" . ", para) # replaces space before and after a full-stop
    para = re.sub(r'[^\x00-\x7f]',r' ',para) # replaces Non-ASCII character with space
    para = no_bracket_regex.sub(" ", para) # replaces content in brackets including the brackets with space
    para = re.sub(' +', ' ', para) # replace multiple-space with single space
    sentences = sent_tokenize(para)
    total_sentences += len(sentences)
    filter2_sentences_extract.append(get_sentences(sentences,tagger,ner_tagger,pos_tagger,search_pattern))
  except:
    print(para)

In [18]:
  def write_to_drive(file_name,filter2_sentences_extract):
    processed_sentences = []
    for item in filter2_sentences_extract:
      if item == []:
        continue
      else:
        for sent in item:
          processed_sentences.append(sent)
    print(len(processed_sentences))
    df = pd.DataFrame({'filtered_sent':processed_sentences})
    df.to_csv('/gdrive/My Drive/NLP-datasets/'+file_name)


In [19]:
write_to_drive('movie_extracted.csv',filter2_sentences_extract)

183
